### KoBERT

출처: https://github.com/SKTBrain/KoBERT

https://colab.research.google.com/github/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb#scrollTo=FW-hpivF5m7Z

- 공개된 코드를 인코더 형식에 맞게 재가공했습니다. multi 인코더나 gpt와 합친 버전 등 앞으로 구현할 방향에 맞게 계속해서 수정하시면 됩니다. 

In [3]:
# !pip uninstall mxnet-cu101
!pip install mxnet-cu101
# !pip uninstall mxnet
# !pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1
!pip install torchvision
!pip install kss


     |███████████████████████████████▌| 834.1MB 1.1MB/s eta 0:00:11tcmalloc: large alloc 1147494400 bytes == 0x396da000 @  0x7f907890e615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 20kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 4.8MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588521 sha256=325534318b8df4923d9f0eac7d89e222e203fe1f81fb9b7444fd89e378d8f023
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 8.0MB

In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qc6n7mwd
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-qc6n7mwd
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12825 sha256=d9c56b56a40356c3d13889912032ee7fc0a5c46bbbb04486f6625caa1ecf947c
  Stored in directory: /tmp/pip-ephem-wheel-cache-u_5ijzzo/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import numpy as np
import time
import kss
import datetime

In [6]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
##GPU 사용 시 #학습시에만 on
#device = torch.device("cuda:0")

##CPU
device = torch.device("cpu")

In [24]:
bertmodel, vocab = get_pytorch_kobert_model()

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

[██████████████████████████████████████████████████]
using cached model


In [63]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [64]:
class EncoderBERT(nn.Module):
    def __init__(self, bert, hidden_size=768, params=None): #hidden_size: encoder output 차원 결정(오리지널 버전에 768로 고정되어있어서 바꾸고 싶으면..대공사를 해야하니 연락주셔요)
        super(EncoderBERT, self).__init__()
        self.bert = bert

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        #attention_mask, token_ids, segment_ids 등은 모두 bert 공부하시면 나오는 개념들이에요. 대부분 사이즈는 (nums of sentences, max_len)로 구성됩니다.
        #모르겠을땐 하나하나 print() 걸어줘서 직접 찍어보는게 이해하는데 제일 빠릅니다.
        attention_mask = self.gen_attention_mask(token_ids, valid_length) #fine-tuning때 필요한거라 안쓰일듯
        sequence_output, pooled_output = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), 
                                                   attention_mask = attention_mask.float().to(token_ids.device))
        return sequence_output, pooled_output
        #sequence_output:(nums of sentences, hidden_size) #pooled_output:(nums of sentences, max_len, hidden_size)
        # 디코더 인풋으로 들어갈 수 있게 형식 맞춰주는 후작업 필요

In [77]:
input_text = input("텍스트를 입력하세요: ")

텍스트를 입력하세요: 안녕하세요. 저는 투빅스 13기 고유경입니다. 중간고사가 방금 끝나서 피곤해요. 내일은 논문을 읽어야 해요. 인생이란 어려우면서도 배울게 많은 여정이랄까요. 여러 생각이 스치는 새벽입니다. 다들 좋은 밤 되시길 바라요.


In [78]:
# kobert는 데이터셋 불러오는 작업이 특이해서 input text를 txt 파일로 변환 후 다시 로딩해야합니다.
# 그래서 위에 dataset 클래스가 새로 정의되어있는거고, 아예 BERTDataset 모듈도 따로 있어요

input_text = kss.split_sentences(input_text) # -> list of sentences
essay = pd.DataFrame(input_text) # -> dataframe
essay['label'] = 1  #의미없음 #이거 안해주면 df 저장 안됨
time_info = datetime.datetime.now().strftime('%Y%m%d%H%M%S') # 파일 이름 설정용 현재시간 로드
save_link = "sample_data/{}.txt".format(time_info) # 현재 위치 폴더에 맞게 경로 재설정해야함
essay.to_csv(save_link, sep='\t', index_label='idx') # csv파일로 저장
#input text data 불러오기
dataset_sentences = nlp.data.TSVDataset(save_link, field_indices=[1, 2], num_discard_samples=1)
data_sentences = BERTDataset(dataset_sentences, 0, 1, tok, 30, True, False) # max_len (30) #예시문장이 다 짧아서 30으로 지정 #자유롭게 바꿔도됨
sentences_dataloader = torch.utils.data.DataLoader(data_sentences, batch_size=len(data_sentences), num_workers=5)

In [79]:
encoder = EncoderBERT(bertmodel).to(device) #위에서 만든 encoder 로드
encoder.eval()

with torch.no_grad(): #파라미터 고정 #만약에 파라미터 학습시키려면 따로 코드를 더 추가해야돼서 연락주세용
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(sentences_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            label = label.long().to(device)
            valid_length = valid_length
            sequence_output, pooled_output = encoder(token_ids, valid_length, segment_ids) #최종 인코더 아웃풋 도출

In [81]:
# 혼란스러울 때, 하나하나 파악하기 위해 찍어보는 코드~
print("1. input_ids: ", token_ids.shape, "\n", token_ids)
print("---------------------")
print("2. segment_ids: ", segment_ids.shape, "\n", segment_ids.long())
print("---------------------")
print("3. 인코더 결과 1) sequence_output: ", sequence_output.shape, "\n", sequence_output)
print("---------------------")
print("4. 인코더 결과 2) pooled_output: ", pooled_output.shape, "\n", pooled_output)

1. input_ids:  torch.Size([7, 30]) 
 tensor([[   2, 3135, 5724, 7814,  517,   54,    3,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1],
        [   2, 3990, 5760, 4762, 6449, 6664,  541, 5561,  993, 7063, 5424, 7139,
          517,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1],
        [   2, 4258, 5439, 6494, 2267, 5550, 1365, 6553, 4909, 5444, 7848, 6999,
          517,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1],
        [   2, 1434, 7126, 7086, 1498, 6235, 3824, 6856, 4998, 6999,  517,   54,
            3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1],
        [   2, 3774, 7107, 3220, 6060, 7005, 6200, 2287, 7013, 5400, 1955, 3298,
         7227, 7096, 6017, 5591, 6999,  517,   54,

---

버트가 좀 복잡하죵.. 코드도 너저분하공.... 그래도 화이팅해봅시당!!